In [311]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, accuracy_score, f1_score, roc_auc_score

In [312]:
data = pd.read_csv("/Users/trishanandakumar/Desktop/BURE/Datasets/loan_data.csv")

In [313]:
education_map = {'High School': 0, 'Associate': 1, 'Bachelor':2, 'Master':3, 'Doctorate':4}

data['person_education'] = data['person_education'].map(education_map)


hot_enc_cols = ['person_gender', 'person_home_ownership','loan_intent', 'previous_loan_defaults_on_file']
data = pd.get_dummies(data, columns=hot_enc_cols,drop_first=True)


to_fix_col = data.select_dtypes(include='bool').columns
data[to_fix_col] = data[to_fix_col].astype(int)


In [314]:
data.head(10)

,person_age,person_education,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status,person_gender_male,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,previous_loan_defaults_on_file_Yes
0,22.0,3,71948.0,0,35000.0,16.02,0.49,3.0,561,1,0,0,0,1,0,0,0,1,0,0
1,21.0,0,12282.0,0,1000.0,11.14,0.08,2.0,504,0,0,0,1,0,1,0,0,0,0,1
2,25.0,0,12438.0,3,5500.0,12.87,0.44,3.0,635,1,0,0,0,0,0,0,1,0,0,0
3,23.0,2,79753.0,0,35000.0,15.23,0.44,2.0,675,1,0,0,0,1,0,0,1,0,0,0
4,24.0,3,66135.0,1,35000.0,14.27,0.53,4.0,586,1,1,0,0,1,0,0,1,0,0,0
5,21.0,0,12951.0,0,2500.0,7.14,0.19,2.0,532,1,0,0,1,0,0,0,0,0,1,0
6,26.0,2,93471.0,1,35000.0,12.42,0.37,3.0,701,1,0,0,0,1,1,0,0,0,0,0
7,24.0,0,95550.0,5,35000.0,11.11,0.37,4.0,585,1,0,0,0,1,0,0,1,0,0,0
8,24.0,1,100684.0,3,35000.0,8.90,0.35,2.0,544,1,0,0,0,1,0,0,0,1,0,0
9,21.0,0,12739.0,0,1600.0,14.74,0.13,3.0,640,1,0,0,1,0,0,0,0,0,1,0


In [315]:
data.describe()
print(data.isnull().sum())

person_age                            0
person_education                      0
person_income                         0
person_emp_exp                        0
loan_amnt                             0
loan_int_rate                         0
loan_percent_income                   0
cb_person_cred_hist_length            0
credit_score                          0
loan_status                           0
person_gender_male                    0
person_home_ownership_OTHER           0
person_home_ownership_OWN             0
person_home_ownership_RENT            0
loan_intent_EDUCATION                 0
loan_intent_HOMEIMPROVEMENT           0
loan_intent_MEDICAL                   0
loan_intent_PERSONAL                  0
loan_intent_VENTURE                   0
previous_loan_defaults_on_file_Yes    0
dtype: int64


In [316]:
scaler = StandardScaler()

In [317]:
threshold = data['loan_amnt'].quantile(0.95)
forget = data[(data['loan_amnt'] > threshold) & (data['previous_loan_defaults_on_file_Yes'] == 1)]
X_forget = forget.drop(columns=['loan_status'])
Y_forget = forget['loan_status']
len(forget)

785

In [318]:
target = LogisticRegression(max_iter=1000)

X = data.drop('loan_status', axis=1)
Y =data['loan_status']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=42)

X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

target.fit(X_train_scale, Y_train)

LogisticRegression(max_iter=1000)

In [319]:
unlearn_scaler = StandardScaler()

In [320]:
keep = data.drop(forget.index)

unlearn = LogisticRegression(max_iter=1000)

X_unlearn = keep.drop('loan_status', axis=1)
Y_unlearn =keep['loan_status']


X_train_unlearn, X_test_unlearn, Y_train_unlearn, Y_test_unlearn = train_test_split(X_unlearn, Y_unlearn, test_size=.2, random_state=42)

X_train_unlearn_scale = unlearn_scaler.fit_transform(X_train_unlearn)
X_test_unlearn_scale = unlearn_scaler.transform(X_test_unlearn)

unlearn.fit(X_train_unlearn_scale , Y_train_unlearn)

LogisticRegression(max_iter=1000)

In [321]:

X_forget_scaled = unlearn_scaler.transform(X_forget)



predictions = target.predict(X_forget_scaled)
predictions_unlearn = unlearn.predict(X_forget_scaled)


print(f"Accuracy: {100*accuracy_score(Y_forget, predictions):.3f}")

print(f"F1: {100*f1_score(Y_forget, predictions, average='weighted'):.3f}")


print("\n")


print(f"Accuracy: {100*accuracy_score(Y_forget, predictions_unlearn):.3f}")

print(f"F1: {100*f1_score(Y_forget, predictions_unlearn, average='weighted'):.3f}")


print("Original model coef:", target.coef_)
print("Unlearned model coef:", unlearn.coef_)



Accuracy: 100.000
F1: 100.000


Accuracy: 100.000
F1: 100.000
Original model coef: [[ 0.16425295 -0.01394109  0.04323043 -0.14578079 -0.63091686  1.00378068
   1.38501938 -0.01359412 -0.4453883   0.01389662  0.01957846 -0.37346444
   0.3567444  -0.34693608 -0.00464929 -0.10747232 -0.26194391 -0.46087829
  -4.49289725]]
Unlearned model coef: [[ 0.17207083  0.00908673  0.05465435 -0.12511053 -0.57766483  0.98540122
   1.35934537 -0.0430488  -0.44604685  0.03459294  0.02712872 -0.36209126
   0.36685877 -0.374495   -0.00828625 -0.12045158 -0.25090438 -0.45774178
  -4.49443447]]
